**The following code load a trained model and predicts on unseen data, outputting a confusion Matrix. Purpose of which we can compare individually trained models and the final ensemble model.**

#COLAB and Libraries and Paths defined
(uncomment if colab used and change paths accordingly in next section)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os 
import random
import cv2
from tqdm import tqdm
import tensorflow as tf
import pandas as pd

from tensorflow.keras import layers
from tensorflow.keras import preprocessing

from google.colab.patches import cv2_imshow

from numpy import loadtxt
from keras.models import load_model
from sklearn import metrics as skcm
from tensorflow.math import confusion_matrix as tfcm

Paths

In [ ]:
model_path = ("/content/drive/MyDrive/FromINKA/Models/densenet_holdout_single_30eps.h5")
# dense = load_model("/content/drive/MyDrive/FromINKA/Models/densenet_holdout_single_30eps.h5")
# resnet = load_model("/content/drive/MyDrive/FromINKA/Models/resnet50v2_holdout_single_30eps.h5")
# efficient = load_model("/content/drive/MyDrive/FromINKA/Models/efficientnet_holdout_single_50eps.h5")


##Unseen data path

DATADIR = "/content/drive/MyDrive/data/comb_dt_splits/prediction/"

FOLDERS = ["B","M"]
CATEGORIES = ["B", "M"] 
NUM_CLASSES = 2
IMG_SIZE = 224
class_names = ["Benign", "Malignant"] 

#Trained Model Load and Data Load into Numpy array with respective labels

Data read function

In [ ]:
def read_data():
    images = [] 
    labels = []
    img_paths = []
    data = []

    for category,folder in zip(CATEGORIES,FOLDERS):     # do B and M 
        path = os.path.join(DATADIR,folder)             # create path to B and M  
        class_num = CATEGORIES.index(category)          # get the classification  (0 or a 1). 0=B and 1=M
        
        for i,img_path in enumerate(tqdm(os.listdir(path))):              # iterate over each image 
            try:
                img_array = cv2.imread(os.path.join(path,img_path))       # convert to array
                img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))   # resize to normalize data size                
                img_paths.append(os.path.join(path,img_path))
                
                data.append([img_array, class_num])  # add this to our training_data
                images.append(img_array)
                labels.append(class_num)
            except Exception as e:                  # in the interest in keeping the output clean...
                pass

    images = np.array(images)                    # output images into np array
    images = np.asarray(images).astype(np.float32)
    labels = np.array(labels)                   # output images into np array
    

    return images,labels

Data read Function call

In [ ]:
images,labels = read_data()

100%|██████████| 686/686 [00:02<00:00, 229.36it/s]


Length check and 0-1 scaling 

In [ ]:
print(len(images))
print(len(labels))
images = images/255 #Uses less computation power

2221
2221


Load model and summary

In [ ]:
selectedModel = load_model("model_path")

selectedModel.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 densenet121 (Functional)    (None, 2)                 7039554   
                                                                 
Total params: 7,039,554
Trainable params: 6,955,906
Non-trainable params: 83,648
_________________________________________________________________


# Predictions

Load each image into Trained model and get prediction 

In [ ]:
benign = 0
malignant = 0
y_predict = []

for img in images:
  
  img = tf.keras.preprocessing.image.img_to_array(img)
  img = tf.expand_dims(img, 0)
  # print(len(img))

  
  ## Prediction step
  predictions = selectedModel.predict(img)
  # print(predictions)

  y_predict.append(predictions.argmax())
  # print(y_predict)
  



  ##for display(not relevant to workflow)
  score = tf.nn.softmax(predictions[0])
  predictedCategory = class_names[np.argmax(score)]
  # print(img)

  if(predictedCategory == "Benign"):
    benign+=1
  elif(predictedCategory == "Malignant"):
    malignant+=1
  else:
      print("bad")

#  break
print(len(y_predict))
print(len(labels))
print("Benign: "+str(benign))
print("Malignant: "+str(malignant))   

2221
2221
Benign: 1463
Malignant: 758


#Confusion Matrix and Classification Metrics

Create Confusion Matrix

In [ ]:
skcm.confusion_matrix(labels, y_predict) 

array([[1388,  147],
       [  75,  611]])

Get Confusion matrix parameters

In [ ]:
tn, fp, fn, tp = skcm.confusion_matrix(labels, y_predict).ravel()
print ("TP: "+ str(tp))
print ("FP: "+ str(fp))
print ("FN: "+ str(fn))
print ("TN: "+ str(tn))

TP: 611
FP: 147
FN: 75
TN: 1388


calculate metrics

In [ ]:
Specificity= tn/(tn+fp)
print ("Specificity:  " +str(Specificity) )

Accuracy= (tp+tn)/(fp+tp+tn+fn)
print ("Accuracy:  " +str(Accuracy) )

Sensitivity= (tp)/(tp+fn)
print ("Sensitivity:  " +str(Sensitivity) )


Specificity:  0.9042345276872964
Accuracy:  0.90004502476362
Sensitivity:  0.89067055393586


#Notes

In [ ]:
print (labels)

[0 0 0 ... 1 1 1]


In [ ]:
print (y_predict)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 

In [ ]:

arr = labels.tolist()
print(arr)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
# tf functions are desinged to work with test data when training is going hense it is called update state(FSM sort) however I m using an unseen dataset 

m = tf.keras.metrics.Accuracy()
m.update_state(labels, y_predict)
Accuracy  =m.result().numpy() 
print ("Accuracy:  " +str(Accuracy) )

m = tf.keras.metrics.Precision()
m.update_state(labels, y_predict)
Precision =m.result().numpy()
print ("Precision:  " +str(Precision) )

m = tf.keras.metrics.Recall()
m.update_state(labels, y_predict)
Recall    =m.result().numpy()
print ("Sensitivity/Recall:  " +str(Recall) )



Accuracy:  0.90004504
Precision:  0.8060686
Sensitivity/Recall:  0.89067054
